# Finalized FLAIR Conversion & Preprocessing Notebook

# USER CONFIG

In [2]:
DATA_ROOT = '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed'
PARTICIPANTS_XLSX = "/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed/participants.xlsx"

# Keep Dataset002 for FLAIR to avoid clashing with T1 Dataset001
OUT_ROOT = '/kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD'

# Pattern hints (extend if needed)
FLAIR_HINTS = ['flair']
MASK_HINTS  = ['roi', 'lesion']  # adapt if your masks follow a different naming

In [1]:
# -*- coding: utf-8 -*-
"""
What this does:
1) Reads participants.xlsx (columns: participant_id, split) to get deterministic train/test.
2) Scans DATA_ROOT for sub-*/anat FLAIR images and lesion masks.
3) Writes:
   - imagesTr/*.nii (FLAIR as _0000.nii) for train
   - imagesTs/*.nii (FLAIR as _0000.nii) for test (no labels required)
   - labelsTr/*.nii (lesion masks) for train only
4) Builds nnU-Net v2 dataset.json with channel_names + labels.
5) Verifies counts + lists unmatched subjects.
6) Resamples labels to exactly match images geometry (order=0) and replaces labelsTr with fixed.

Notes:
- Saves uncompressed .nii (no .gz) for compatibility.
- Subjects without masks are allowed in test (images only), but excluded from train.
"""

import os, json, shutil
from pathlib import Path
import pandas as pd
import nibabel as nib

# Try canonical orientation helper for safer header/orientation handling
try:
    from nibabel.processing import as_closest_canonical
except Exception:
    def as_closest_canonical(img): return img

# Optional for late-stage mask<->image alignment
from nibabel.processing import resample_from_to

In [3]:
import pandas as pd
df = pd.read_excel(PARTICIPANTS_XLSX)
print(df.columns.tolist())
df.head()

['participant_id', 'group', 'sex', 'age_epilepsyonset', 'age_scan', 'mri_diagnosis', 'map', 'spect', 'pet', 'invasive_EEG', 'op', 'age_op', 'histopathology', '1year_outcome', 'latest_outcome', 'time_follow-up', 'hemisphere', 'lobe', 'split']


/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()


,participant_id,group,sex,age_epilepsyonset,age_scan,mri_diagnosis,map,spect,pet,invasive_EEG,op,age_op,histopathology,1year_outcome,latest_outcome,time_follow-up,hemisphere,lobe,split
0,sub-00001,fcd,M,4.0,4,suspicion,1.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,R,FL,train
1,sub-00002,hc,F,NaN,12,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train
2,sub-00003,fcd,M,1.0,11,suspicion,0.0,0.0,0.0,0.0,0,NaN,NaN,NaN,NaN,NaN,R,PL,train
3,sub-00004,fcd,M,1.0,6,suspicion,0.0,1.0,0.0,1.0,1,6.0,IIb,IA,IA,2.0,L,FL,test
4,sub-00005,hc,F,NaN,12,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,train


# OUTPUT FOLDERS 

In [4]:
imagesTr_dir = os.path.join(OUT_ROOT, 'imagesTr')
imagesTs_dir = os.path.join(OUT_ROOT, 'imagesTs')
labelsTr_dir = os.path.join(OUT_ROOT, 'labelsTr')

# Clear previous outputs to avoid duplicates and stale files
if os.path.exists(OUT_ROOT):
    shutil.rmtree(OUT_ROOT)
os.makedirs(imagesTr_dir, exist_ok=True)
os.makedirs(imagesTs_dir, exist_ok=True)
os.makedirs(labelsTr_dir, exist_ok=True)

# HELPERS

In [5]:
def is_flair(filename: str) -> bool:
    fn = filename.lower().strip()
    if not fn.endswith('.nii') and not fn.endswith('.nii.gz'):
        return False
    return any(h in fn for h in FLAIR_HINTS)

def is_mask(filename: str) -> bool:
    fn = filename.lower().strip()
    if not (fn.endswith('.nii') or fn.endswith('.nii.gz')):
        return False
    return any(h in fn for h in MASK_HINTS)

def normalize_id(s: str) -> str:
    """
    Normalize IDs to digits without leading zeros.
    Examples:
      'sub-00007' -> '7'
      '0003'     -> '3'
      '7'        -> '7'
    """
    if s is None:
        return '0'
    s = str(s).strip()
    if s.lower().startswith('sub-'):
        s = s[4:]
    # keep only digits
    digits = ''.join(ch for ch in s if ch.isdigit())
    return digits.lstrip('0') or '0'

def scan_subject(sub_path: str):
    """
    Return dict with: case, case_norm, flair (path or None), mask (path or None)
    Picks ONE flair and ONE mask if multiple exist (first match in sorted order).
    """
    anat = os.path.join(sub_path, 'anat')
    if not os.path.isdir(anat):
        return None
    files = sorted(os.listdir(anat))
    flair_path = None
    mask_path  = None
    for f in files:
        if is_flair(f) and flair_path is None:
            flair_path = os.path.join(anat, f)
        if is_mask(f) and mask_path is None:
            mask_path = os.path.join(anat, f)
    if flair_path is None:
        return None
    case_name = os.path.basename(sub_path)  # e.g., sub-00074
    return {'case': case_name, 'case_norm': normalize_id(case_name), 'flair': flair_path, 'mask': mask_path}

def save_uncompressed(imgobj: nib.Nifti1Image, out_path_nii: str):
    """Force-save uncompressed .nii (remove any .nii.gz if present)."""
    if out_path_nii.endswith('.gz'):
        out_path_nii = out_path_nii[:-3]
    # remove conflicting leftovers
    if os.path.exists(out_path_nii + '.gz'):
        os.remove(out_path_nii + '.gz')
    if os.path.exists(out_path_nii):
        os.remove(out_path_nii)
    nib.save(imgobj, out_path_nii)

def write_case(case_dict: dict, in_train: bool):
    """Write image (+ label if train) in nnU-Net folder names."""
    case = case_dict['case']
    flair_src = case_dict['flair']
    mask_src  = case_dict['mask']

    # Load FLAIR and save as _0000.nii
    img = nib.load(flair_src)
    img = as_closest_canonical(img)
    img_dst = os.path.join(imagesTr_dir if in_train else imagesTs_dir, f"{case}_0000.nii")
    save_uncompressed(img, img_dst)

    # If in train, we need a label
    if in_train:
        if mask_src is None:
            print(f"⚠️ Skipping label for {case} (no mask). This subject cannot be in train without a mask.")
        else:
            lbl = nib.load(mask_src)
            lbl = as_closest_canonical(lbl)
            lbl_dst = os.path.join(labelsTr_dir, f"{case}.nii")
            save_uncompressed(lbl, lbl_dst)

# READ EXCEL + BUILD SPLIT MAP

In [6]:
df = pd.read_excel(PARTICIPANTS_XLSX)
required_cols = {'participant_id', 'split'}
if not required_cols.issubset(set(df.columns.str.lower())):
    # try to be resilient with column name casing
    cols_map = {c.lower(): c for c in df.columns}
    if not required_cols.issubset(set(cols_map.keys())):
        raise ValueError("Excel must contain 'participant_id' and 'split' columns. Found: %s" % list(df.columns))

    pid_col   = cols_map['participant_id']
    split_col = cols_map['split']
else:
    # original
    pid_col, split_col = 'participant_id', 'split'

df[pid_col] = df[pid_col].astype(str).str.strip()
df[split_col] = df[split_col].astype(str).str.strip().str.lower()

split_map = {}
for _, row in df.iterrows():
    pid_norm = normalize_id(row[pid_col])
    split_map[pid_norm] = row[split_col]  # 'train' or 'test'

# SCAN DATASET

In [7]:
subs = [os.path.join(DATA_ROOT, d) for d in sorted(os.listdir(DATA_ROOT)) if d.startswith('sub-')]
cases_all = []
for s in subs:
    c = scan_subject(s)
    if c is not None:
        cases_all.append(c)

print(f"Found {len(cases_all)} FLAIR subjects. Example entries:\n", cases_all[:6])

Found 170 FLAIR subjects. Example entries:
 [{'case': 'sub-00001', 'case_norm': '1', 'flair': '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed/sub-00001/anat/sub-00001_acq-T2sel_FLAIR.nii', 'mask': '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed/sub-00001/anat/sub-00001_acq-T2sel_FLAIR_roi.nii'}, {'case': 'sub-00002', 'case_norm': '2', 'flair': '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed/sub-00002/anat/sub-00002_acq-T2sel_FLAIR.nii', 'mask': None}, {'case': 'sub-00003', 'case_norm': '3', 'flair': '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed/sub-00003/anat/sub-00003_acq-T2sel_FLAIR.nii', 'mask': '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed/sub-00003/anat/sub-00003_acq-T2sel_FLAIR_roi.nii'}, {'case': 'sub-00004', 'case_norm': '4', 'flair': '/kaggle/input/organized-bonn-fcd-ii-epilepsy-mri-dataset/bonn_fcd_fixed/sub-00004/anat/sub-00004_acq-T2sel_FLAIR.nii'

# BUILD TRAIN/TEST BY EXCEL
 - Train: needs mask (skip if mask missing)
 - Test: image required (mask optional) 

In [8]:
train_cases = []
test_cases  = []
unmatched   = []  # subjects present in data but not in Excel or invalid split values

for c in cases_all:
    key = c['case_norm']
    if key not in split_map:
        unmatched.append((c, "not present in Excel split"))
        continue
    s = split_map[key]
    if s == 'train':
        if c['mask'] is not None:
            train_cases.append(c)
        else:
            unmatched.append((c, "train in Excel but mask missing -> excluded from train"))
    elif s == 'test':
        test_cases.append(c)  # mask optional
    else:
        unmatched.append((c, f"invalid split value: {s}"))

# Basic sanity
train_ids = set([c['case'] for c in train_cases])
test_ids  = set([c['case'] for c in test_cases])
overlap = train_ids.intersection(test_ids)
if overlap:
    raise RuntimeError(f"Overlap between train and test assignments: {overlap}")

print(f"Assigned from Excel -> Train: {len(train_cases)} | Test: {len(test_cases)} | Unmatched: {len(unmatched)}")
if unmatched:
    print("Examples of unmatched/excluded (up to 10):")
    for c, reason in unmatched[:10]:
        print(f"  {c['case']}  normalized={c['case_norm']}  reason={reason}")

Assigned from Excel -> Train: 57 | Test: 28 | Unmatched: 85
Examples of unmatched/excluded (up to 10):
  sub-00002  normalized=2  reason=train in Excel but mask missing -> excluded from train
  sub-00005  normalized=5  reason=train in Excel but mask missing -> excluded from train
  sub-00007  normalized=7  reason=train in Excel but mask missing -> excluded from train
  sub-00008  normalized=8  reason=train in Excel but mask missing -> excluded from train
  sub-00011  normalized=11  reason=train in Excel but mask missing -> excluded from train
  sub-00012  normalized=12  reason=train in Excel but mask missing -> excluded from train
  sub-00013  normalized=13  reason=train in Excel but mask missing -> excluded from train
  sub-00017  normalized=17  reason=train in Excel but mask missing -> excluded from train
  sub-00019  normalized=19  reason=train in Excel but mask missing -> excluded from train
  sub-00021  normalized=21  reason=train in Excel but mask missing -> excluded from train


# WRITE FILES

In [9]:
for c in train_cases:
    write_case(c, in_train=True)
for c in test_cases:
    write_case(c, in_train=False)

# Final cleanup: ensure no .nii.gz leftovers (we only want .nii)
for root, dirs, files in os.walk(OUT_ROOT):
    for fn in files:
        if fn.endswith('.nii.gz'):
            try:
                os.remove(os.path.join(root, fn))
            except Exception:
                pass

# dataset.json (nnU-Net v2 style)

In [10]:
dataset_v2 = {
    "dataset_name": "BonnFCD_FLAIR",
    "description": "Bonn FCD II Epilepsy MRI dataset (FLAIR + ROI masks, Excel-driven split)",
    "reference": "Bonn University Hospital",
    "licence": "see data provider",
    "release": "1.0",
    "channel_names": {
        "0": "FLAIR"
    },
    "labels": {
        "background": 0,
        "lesion": 1
    },
    "numTraining": len(train_cases),
    "numTest": len(test_cases),
    "file_ending": ".nii",
    "overwrite_image_reader_writer": "SimpleITKIO",
    # The v2 fields below are optional for training; training/test lists are handled by nnUNetv2 conversion
}

with open(os.path.join(OUT_ROOT, 'dataset.json'), 'w') as f:
    json.dump(dataset_v2, f, indent=2)

print("\n✅ dataset.json written at:", os.path.join(OUT_ROOT, 'dataset.json'))


✅ dataset.json written at: /kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD/dataset.json


# VERIFY COUNTS


In [11]:
print("\n=== Verification summary ===")
print("Excel train count (any):", sum(1 for v in split_map.values() if v=='train'))
print("Excel test count (any):", sum(1 for v in split_map.values() if v=='test'))
print("Found FLAIR subjects:", len(cases_all))
print("Assigned train_cases (with masks):", len(train_cases))
print("Assigned test_cases:", len(test_cases))
print("Files in imagesTr:", len(os.listdir(imagesTr_dir)))
print("Files in imagesTs:", len(os.listdir(imagesTs_dir)))
print("Files in labelsTr:", len(os.listdir(labelsTr_dir)))

# Show a few for quick spot-check
def head_n(p, n=10):
    try:
        return sorted(os.listdir(p))[:n]
    except Exception:
        return []

print("\nFirst imagesTr:", head_n(imagesTr_dir))
print("First imagesTs:", head_n(imagesTs_dir))
print("First labelsTr:", head_n(labelsTr_dir))


=== Verification summary ===
Excel train count (any): 142
Excel test count (any): 28
Found FLAIR subjects: 170
Assigned train_cases (with masks): 57
Assigned test_cases: 28
Files in imagesTr: 57
Files in imagesTs: 28
Files in labelsTr: 57

First imagesTr: ['sub-00001_0000.nii', 'sub-00003_0000.nii', 'sub-00010_0000.nii', 'sub-00014_0000.nii', 'sub-00015_0000.nii', 'sub-00016_0000.nii', 'sub-00018_0000.nii', 'sub-00024_0000.nii', 'sub-00027_0000.nii', 'sub-00033_0000.nii']
First imagesTs: ['sub-00004_0000.nii', 'sub-00006_0000.nii', 'sub-00009_0000.nii', 'sub-00020_0000.nii', 'sub-00032_0000.nii', 'sub-00034_0000.nii', 'sub-00043_0000.nii', 'sub-00048_0000.nii', 'sub-00061_0000.nii', 'sub-00062_0000.nii']
First labelsTr: ['sub-00001.nii', 'sub-00003.nii', 'sub-00010.nii', 'sub-00014.nii', 'sub-00015.nii', 'sub-00016.nii', 'sub-00018.nii', 'sub-00024.nii', 'sub-00027.nii', 'sub-00033.nii']


# Post-step: ensure labels have exact same geometry as images
## (Resample masks to image geometry and replace labelsTr with fixed)

In [13]:
fixed_labels = os.path.join(OUT_ROOT, 'labelsTr_fixed')
os.makedirs(fixed_labels, exist_ok=True)

for lbl_name in sorted(os.listdir(labelsTr_dir)):
    if not lbl_name.endswith('.nii'):
        continue
    case_id = lbl_name.replace('.nii', '')
    img_path = os.path.join(imagesTr_dir, f"{case_id}_0000.nii")
    seg_path = os.path.join(labelsTr_dir, lbl_name)
    if not os.path.exists(img_path):
        print(f"⚠️ Skipping {case_id}: matching FLAIR image not found.")
        continue
    img = nib.load(img_path)
    seg = nib.load(seg_path)
    seg_resampled = resample_from_to(seg, img, order=0)  # preserve label integers
    nib.save(seg_resampled, os.path.join(fixed_labels, lbl_name))

# Replace labelsTr with fixed
shutil.rmtree(labelsTr_dir)
shutil.move(fixed_labels, labelsTr_dir)
print("✅ Labels resampled & labelsTr replaced with fixed labels.")

print("\n🎯 FLAIR dataset ready at:", OUT_ROOT)
print("   You can now run:")
print("   1) pip install nnunetv2  (if not installed)")
print("   2) export nnUNet_raw=/kaggle/working/nnUNet_raw_data")
print("      export nnUNet_preprocessed=/kaggle/working/nnUNet_preprocessed")
print("      export nnUNet_results=/kaggle/working/nnUNet_results")
print("   3) nnUNetv2_plan_and_preprocess -d 002 --verify_dataset_integrity")

✅ Labels resampled & labelsTr replaced with fixed labels.

🎯 FLAIR dataset ready at: /kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD
   You can now run:
   1) pip install nnunetv2  (if not installed)
   2) export nnUNet_raw=/kaggle/working/nnUNet_raw_data
      export nnUNet_preprocessed=/kaggle/working/nnUNet_preprocessed
      export nnUNet_results=/kaggle/working/nnUNet_results
   3) nnUNetv2_plan_and_preprocess -d 002 --verify_dataset_integrity


In [14]:
!pip install nnunetv2

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 kB 6.8 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.0/77.0 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.8/44.8 kB 1.4 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 5.8 MB/s eta 0:00:000:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 47.0 MB/s eta 0:00:00:00:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 28.9 MB/s eta 0:00:00
   ━━━━━━━━━━━

In [15]:
import os
# Set nnU-Net expected env paths
os.environ["nnUNet_raw"] = "/kaggle/working/nnUNet_raw_data"
os.environ["nnUNet_preprocessed"] = "/kaggle/working/nnUNet_preprocessed"
os.environ["nnUNet_results"] = "/kaggle/working/nnUNet_results"

print("nnUNet_raw       =", os.environ["nnUNet_raw"])
print("nnUNet_preprocessed =", os.environ["nnUNet_preprocessed"])
print("nnUNet_results   =", os.environ["nnUNet_results"])

nnUNet_raw       = /kaggle/working/nnUNet_raw_data
nnUNet_preprocessed = /kaggle/working/nnUNet_preprocessed
nnUNet_results   = /kaggle/working/nnUNet_results


# Verify dataset integrity & plan/preprocess

In [16]:
# Show basic tree & counts
import os, json, textwrap, subprocess, glob
root = "/kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD"
for sub in ["imagesTr", "labelsTr", "imagesTs"]:
    p = os.path.join(root, sub)
    print(sub, "->", len(os.listdir(p)) if os.path.isdir(p) else 0)

!head -n 60 /kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD/dataset.json || true

imagesTr -> 57
labelsTr -> 57
imagesTs -> 28
{
  "dataset_name": "BonnFCD_FLAIR",
  "description": "Bonn FCD II Epilepsy MRI dataset (FLAIR + ROI masks, Excel-driven split)",
  "reference": "Bonn University Hospital",
  "licence": "see data provider",
  "release": "1.0",
  "channel_names": {
    "0": "FLAIR"
  },
  "labels": {
    "background": 0,
    "lesion": 1
  },
  "numTraining": 57,
  "numTest": 28,
  "file_ending": ".nii",
  "overwrite_image_reader_writer": "SimpleITKIO"
}

In [17]:
from pathlib import Path
import nibabel as nib, json, os

root = Path("/kaggle/working/nnUNet_raw_data/Dataset002_BonnFCD")
assert root.exists(), "Dataset folder missing"
for d in ["imagesTr","labelsTr","imagesTs"]:
    p = root/d
    assert p.exists(), f"Missing {d}"
    
# file ending check
with open(root/"dataset.json") as f:
    dj = json.load(f)
assert dj.get("file_ending")==".nii", "dataset.json file_ending must be .nii"

# name pairing check
bad = []
for lbl in (root/"labelsTr").glob("*.nii"):
    img = root/"imagesTr"/f"{lbl.stem}_0000.nii"
    if not img.exists():
        bad.append(lbl.name)
assert not bad, f"Missing matching images for labels: {bad[:10]}"

# geometry check on a few
checked=0
for lbl in list((root/"labelsTr").glob("*.nii"))[:5]:
    img = nib.load(str(root/"imagesTr"/f"{lbl.stem}_0000.nii"))
    seg = nib.load(str(lbl))
    assert img.shape==seg.shape, f"Shape mismatch {lbl.stem}: {img.shape} vs {seg.shape}"
    checked+=1
print(f"Preflight OK. Checked {checked} label-image pairs.")


Preflight OK. Checked 5 label-image pairs.


In [18]:
# Remove any previous preprocessing cache to avoid stale fingerprints:
!rm -rf /kaggle/working/nnUNet_preprocessed/Dataset002_BonnFCD

# Verify integrity and run planning/preprocessing

In [19]:
!nnUNetv2_plan_and_preprocess -d 002 --verify_dataset_integrity

Fingerprint extraction...
Dataset002_BonnFCD
Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> reader/writer

####################
verify_dataset_integrity Done. 
If you didn't see any error messages then your dataset is most likely OK!
####################

Using <class 'nnunetv2.imageio.simpleitk_reader_writer.SimpleITKIO'> reader/writer
100%|███████████████████████████████████████████| 57/57 [00:30<00:00,  1.85it/s]
Experiment planning...

############################
INFO: You are using the old nnU-Net default planner. We have updated our recommendations. Please consider using those instead! Read more here: https://github.com/MIC-DKFZ/nnUNet/blob/master/documentation/resenc_presets.md
############################

Attempting to find 3d_lowres config. 
Current spacing: [1.03 1.03 1.03]. 
Current patch size: (160, 160, 96). 
Current median shape: [247.57281553 247.57281553 155.33980583]
Dropping 3d_lowres config because the image size difference to 3d_fullres is to